In [47]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import re
from urllib.request import urlopen


In [48]:
card_names = []
card_imgs = []
card_summary = []
card_benefit = []

In [49]:
# 카드 혜택 요약 정보 크롤링 함수
def crawl_card_summary(card_url):
    html = urlopen(card_url)
    soup = BeautifulSoup(html, 'html.parser')

    # 카드 요약 정보 딕셔너리
    card_summary = {
        "Card Summary": []
    }

    # 카드 요약 크롤링 코드
    ul_dot_elements = soup.find_all('ul', {'class': 'txt-list-dot'})
    if ul_dot_elements:
        for ul_dot_element in ul_dot_elements:
            for text in ul_dot_element.stripped_strings:
                card_summary['Card Summary'].append(text)

    return card_summary

In [50]:
# 카드 혜택 상세 정보 크롤링 함수
def crawl_card_benefit(card_url):
    html = urlopen(card_url)
    soup = BeautifulSoup(html, 'html.parser')

    # 카드 상세 정보 딕셔너리
    card_info = {
        "Card Benefit": []
    }

    # 카드 혜택 크롤링 코드
    container_box = soup.find('section', class_='tab-container')
    div_boxes = container_box.find_all('div', class_='list_box_nt')
    for div_box in div_boxes:
        h4_element = div_box.find_previous_sibling('h4')
        if h4_element:
            card_info['Card Benefit'].append("\n<"+h4_element.text.replace('\\xa','').replace('\n','')+">")
        titles = div_box.find_all('h5', class_='tit04') # 타이틀 리스트
        tables = div_box.find_all('div', class_='table_col') # 표 리스트
        uls = div_box.find_all('ul', class_='txt_list') # ul 리스트
        service_infos = uls[0].find_all('span', class_='txt_bar')
        if service_infos[0] and (service_infos[0].text.strip()[-2:] == "할인" or service_infos[0].text.strip()[-3:] == "할인)" 
                                    or service_infos[0].text.strip()[-8:-6] == "할인" or service_infos[0].text.strip()[-2:] == "적립" 
                                    or service_infos[0].text.strip()[-11:-9] == "할인" or service_infos[0].text.strip()[-14:-12] == "할인" 
                                    or service_infos[0].text.strip()[-2:] == "무료") or service_infos[0].text.strip()[-12:-10] == "할인":
            num = 0 # 타이틀 번호
            table_num = 0 # 표 번호
            uls_num = 1
            # 서비스 안내
            if titles[num].text == "서비스안내":
                title1 = titles[num].text
                num = num+1
                # card_info['Card Benefit'].append(str(num) + ") " + title1)
                card_info['Card Benefit'].append("["+title1+"]")
                card_info['Card Benefit'].append(uls[0].text.replace('\n', ''))
                next_ul_check = uls[uls_num]
                if next_ul_check.text.strip().startswith("자세한 내용"):
                    uls_num += 1
            
            # 대상점
            if titles[num].text == "대상점":
                title2 = titles[num].text
                num = num+1
                table1 = tables[table_num]
                table_num = table_num+1

                target_title = table1.find('thead')
                target_shops = table1.find('tbody')
                # card_info['Card Benefit'].append(str(num) + ") " + title2)
                card_info['Card Benefit'].append("###[" + title2 + "]")
                if target_title:
                    card_info['Card Benefit'].append(target_title.text.replace('\n', ''))
                card_info['Card Benefit'].append(target_shops.text.replace('\n', ''))

            # 연간 이용금액대별 적립률
            if titles[num].text == "연간 이용금액대별 적립률":
                title2 = titles[num].text
                num = num+1
                # card_info['Card Benefit'].append(str(num) + ") " + title2)
                card_info['Card Benefit'].append("###[" + title2 + "]")
                table2 = tables[table_num]
                table_num = table_num+1                   
                table_title = table2.find('caption').text.replace('\n', '') # 테이블 이름
                card_info['Card Benefit'].append(table_title)
                # 테이블 요소 개수 count
                table_tr = table2.find_all('tr')
                table2_tr_num = len(table_tr)
                # 구분 list
                table2_discount_prices = table_tr[0].find_all('th')
                # 기본 적립 list
                table2_tbody = table2.find('tbody')
                table2_tbody_tr = table2_tbody.find_all('tr')
                table2_basic_points = table2_tbody_tr[0].find_all('td')
                # 추가 적립 list
                table2_plus_points = table2_tbody_tr[1].find_all('td')
                for i in range(min(len(table2_discount_prices), len(table2_basic_points), len(table2_plus_points))):
                    table2_price = table2_discount_prices[i].text.replace('\n', '')
                    table2_basic_point = table2_basic_points[i].text.replace('\n', '')
                    table2_plus_point = table2_plus_points[i].text.replace('\n', '')
                    card_info['Card Benefit'].append(f"{table2_price} 기본 적립:{table2_basic_point} 추가 적립:{table2_plus_point}")
            

            # 이용한도 및 조건
            if titles[num].text == "이용한도 및 조건":
                title2 = titles[num].text
                num = num+1
                # card_info['Card Benefit'].append(str(num) + ") " + title2)
                card_info['Card Benefit'].append("###[" + title2 + "]")
                max_and_conditions = uls[uls_num].find_all('li')
                uls_num = uls_num+1
                for max_and_condition in max_and_conditions:
                    card_info['Card Benefit'].append(max_and_condition.text.replace('\n', '')) 


            # 할인기준
            if titles[num].text == "할인기준":
                title3 = titles[num].text
                num = num+1
                table2 = tables[table_num]
                table_num = table_num+1
                # card_info['Card Benefit'].append(str(num) + ") " + title3)
                card_info['Card Benefit'].append("###[" + title3 +"]")
                table_title = table2.find('caption').text # 테이블 이름
                card_info['Card Benefit'].append(table_title.replace('\n', ''))
                # 테이블 요소 개수 count
                table_tr = table2.find_all('tr')
                table2_tr_num = len(table_tr)
                # 할인 기준 금액 list
                table2_discount_prices = table_tr[1].find_all('th')
                # 할인한도 금액 list
                table2_tbody = table2.find('tbody')
                table2_contents = table2_tbody.find_all('tr')

                for table2_discount_price in table2_discount_prices:
                    card_info['Card Benefit'].append(table2_discount_price.text.replace('\n', ''))

                for table2_content in table2_contents:
                    card_info['Card Benefit'].append(table2_content.text.replace('\n', ''))

            # 전월 이용금액대별 통합 월 할인한도
            if titles[num].text == "전월 이용금액대별 통합 월 할인한도":
                title3 = titles[num].text
                num = num+1
                table2 = tables[table_num]
                table_num = table_num+1
                # card_info['Card Benefit'].append(str(num) + ") " + title3)
                card_info['Card Benefit'].append("###[" + title3 + "]")
                table_title = table2.find('caption').text.replace('\n', '') # 테이블 이름
                card_info['Card Benefit'].append(table_title)
                # 테이블 요소 개수 count
                table_tr = table2.find_all('tr')
                table2_tr_num = len(table_tr)
                # 할인 기준 금액 list
                table2_discount_prices = table_tr[0].find_all('th')
                # 할인한도 금액 list
                table2_tbody = table2.find('tbody')
                table2_contents = table2_tbody.find_all('td')

                for i in range(min(len(table2_discount_prices), len(table2_contents))):
                    table2_price = table2_discount_prices[i].text.replace('\n', '')
                    table2_content = table2_contents[i].text.replace('\n', '')
                    card_info['Card Benefit'].append(table2_price + " " + table2_content)

            # 전월 이용금액대별 할인율 및 통합 월 할인한도
            if titles[num].text == "전월 이용금액대별 할인율 및 통합 월 할인한도":
                title4 = titles[num].text
                num = num+1
                table3 = tables[table_num]
                table_num = table_num+1
                # card_info['Card Benefit'].append(str(num) + ") " + title4)
                card_info['Card Benefit'].append("###[" + title4 + "]")
                table_title = table3.find('caption').text # 테이블 이름     
                card_info['Card Benefit'].append(table_title.replace('\n', ''))
                table_element1 = table3.find('thead')
                table_use_prices = table_element1.find_all('th', scope='col') # 전월 이용금액 리스트
                table3_num = len(table_use_prices) # 테이블 요소 개수
                table_element2 = table3.find('tbody')
                table_discount_contents = table_element2.find_all('tr')
                table_discount_percents = table_discount_contents[0] # 할인율 리스트
                table_discount_maxs = table_discount_contents[1] # 할인 한도 리스트
                table_discount_percents_detail = table_discount_percents.find_all('span', class_='ls0') # 할인율 리스트
                table_discount_maxs_detail = table_discount_maxs.find_all('td', class_='al_r')
                for i in range(table3_num-1):
                    table_price = table_use_prices[i+1].text
                    table_discount_percent = table_discount_percents_detail[i].text
                    table_discount_max = table_discount_maxs_detail[i].text
                    card_info['Card Benefit'].append(table_price + " 할인율:" + table_discount_percent.replace('\n', '') + " 할인 한도:" + table_discount_max.replace('\n', ''))

            # 이용방법
            if num < len(titles) and titles[num].text == "이용방법":
                title5 = titles[num].text
                num = num+1
                exceptions = uls[uls_num].find_all('li')
                uls_num = uls_num+1
                # card_info['Card Benefit'].append(str(num) + ") " + title5)
                card_info['Card Benefit'].append("###" + title5+"]")
                for exception in exceptions:
                    card_info['Card Benefit'].append(exception.text.replace('\n', ''))

            # 적립기준
            if titles[num].text == "적립기준":
                title2 = titles[num].text
                num = num+1
                table1 = tables[table_num]
                table_num = table_num+1

                target_title = table1.find('thead')
                target_shops = table1.find('tbody')
                # card_info['Card Benefit'].append(str(num) + ") " + title2)
                card_info['Card Benefit'].append("###["+title2+"]")
                if target_title:
                    card_info['Card Benefit'].append(target_title.text.replace('\n', ''))
                card_info['Card Benefit'].append(target_shops.text.replace('\n', ''))

            # 이용조건
            if (titles[num].text == "이용조건") or (titles[num].text == "이용 조건 "):
                title4 = titles[num].text
                num = num+1
                condition_infos = uls[uls_num].find_all('li')
                condition_uls = uls[uls_num].find_all('ul', class_='txt_list')
                if condition_uls:
                    uls_num = uls_num + 1
                uls_num = uls_num+1
                # card_info['Card Benefit'].append(str(num) + ") " + title4)
                card_info['Card Benefit'].append("###[" + title4 + "]")
                for condition_info in condition_infos:
                    card_info['Card Benefit'].append(condition_info.text.replace('\n', ''))

            # 적립 제외 대상
            if titles[num].text == "적립 제외 대상":
                title5 = titles[num].text
                num = num+1
                exceptions = uls[uls_num].find_all('li')
                uls_num = uls_num+1
                # card_info['Card Benefit'].append(str(num) + ") " + title5)
                card_info['Card Benefit'].append("###[" + title5 +"]")
                for exception in exceptions:
                    card_info['Card Benefit'].append(exception.text.replace('\n', ''))
                
            # 할인 제외 대상
            if num < len(titles) and titles[num].text == "할인 제외 대상":
                title5 = titles[num].text
                num = num+1
                exceptions = uls[uls_num].find_all('li')
                uls_num = uls_num+1
                # card_info['Card Benefit'].append(str(num) + ") " + title5)
                card_info['Card Benefit'].append("###[" + title5+"]")
                for exception in exceptions:
                    card_info['Card Benefit'].append(exception.text.replace('\n', ''))

            # 부가세환급 편의지원서비스
            if num < len(titles) and titles[num].text == "부가세환급 편의지원서비스":
                title5 = titles[num].text
                num = num+1
                exceptions = uls[uls_num].find_all('li')
                uls_num = uls_num+1
                # card_info['Card Benefit'].append(str(num) + ") " + title5)
                card_info['Card Benefit'].append("###[" + title5+"]")
                for exception in exceptions:
                    card_info['Card Benefit'].append(exception.text.replace('\n', ''))

            # 전월 이용금액 기준
            if num < len(titles) and titles[num].text == "전월 이용금액 기준":
                title5 = titles[num].text
                num = num+1
                exceptions = uls[uls_num].find_all('li')
                uls_num = uls_num+1
                # card_info['Card Benefit'].append(str(num) + ") " + title5)
                card_info['Card Benefit'].append("###[" + title5 +"]")
                for exception in exceptions:
                    card_info['Card Benefit'].append(exception.text.replace('\n', ''))

            # 기타
            if num < len(titles) and titles[num].text == "기타":
                title5 = titles[num].text
                num = num+1
                exceptions = uls[uls_num].find_all('li')
                uls_num = uls_num+1
                # card_info['Card Benefit'].append(str(num) + ") " + title5)
                card_info['Card Benefit'].append("###[" + title5+"]")
                for exception in exceptions:
                    card_info['Card Benefit'].append(exception.text.replace('\n', ''))

    return card_info

In [51]:
# 크롤링 정보 정리 (특수문자 제거 및 줄바꿈 제거)
def remove_special_characters(text):

    # 특수문자 제거
    text_str = str(text)
    cleaned_text = re.sub(r'[^\w가-힣\s.%~\n#]', '', text_str)


    # 여러 개의 공백을 하나의 공백으로 변환
    cleaned_text = ' '.join(cleaned_text.split())

    return cleaned_text

In [55]:
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 카드 리스트 페이지 URL
card_list_url = "https://www.samsungcard.com/home/card/cardinfo/PGHPPDCCardCardinfoRecommendPC001"
# driver.get(card_list_url)
# driver.implicitly_wait(10)
# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')
html = urlopen(card_list_url)
soup = BeautifulSoup(html, 'html.parser')

# 카드 리스트 및 혜택 정보 담는 객체
card_list = []

# 카드 리스트 링크 가져오기
card_lists = soup.find_all('ul', class_='lists')
card_list = card_lists[1]
card_elements = card_list.find_all('li')

for card_element in card_elements:

    # 카드 이름 정보
    card_name = card_element.find('div', class_='tit-h4')
    print(card_name.text)
    card_names.append(card_name.text)

    # 카드 이미지 정보
    card_img = card_element.find('img')
    card_img_url = card_img.get('src')
    print(card_img_url)
    card_imgs.append(card_img_url)

    # 카드 고유 번호 추출
    card_url_code = card_img_url[-11:-4]
    card_url = f"https://www.samsungcard.com/home/card/cardinfo/PGHPPCCCardCardinfoDetails001?code={card_url_code}"

    # 카드 혜택 요약 정보
    card_summay_benefit = crawl_card_summary(card_url)
    clean_card_summary_benefit = remove_special_characters(card_summay_benefit)
    print(clean_card_summary_benefit)
    card_summary.append(clean_card_summary_benefit)

    # 카드 혜택 상세 정보
    card_detail_benefit = crawl_card_benefit(card_url)
    # clean_card_detail_benefit = remove_special_characters(card_detail_benefit)
    # clean_card_detail_benefit = ' '.join(str(card_detail_benefit).split())

    clean_card_detail_benefit = ''.join(''.join(benefit) for benefit in card_detail_benefit['Card Benefit'])

    print(clean_card_detail_benefit)
    card_benefit.append(clean_card_detail_benefit)
    print()
    print()

# driver.quit()

삼성 iD ON 카드
https://static11.samsungcard.com/wcms/home/scard/image/personal/IDON_AAP1731.gif
Card Summary 카드 디자인 소개 많이 쓰는 영역 30% 자동 맞춤 할인 교통이동통신스트리밍 10% 결제일할인 온라인 간편결제해외 3%1% 결제일할인

<많이 쓰는 영역 30% 자동 맞춤 할인>[서비스안내]커피전문점·배달앱·델리 영역 중 월 이용금액이 가장 큰 1개 영역에 대해 30% 결제일할인온라인 간편결제·해외 3%·1% 할인 혜택과 중복 적용###[대상점]영역 할인 대상커피전문점스타벅스, 이디야커피, 커피빈, 투썸플레이스, 블루보틀배달앱배달의민족, 요기요델리쉐이크쉑, 써브웨이, 파리바게뜨, 배스킨라빈스, 던킨###[이용조건]월 할인한도 : 10,000원전월 이용금액 30만원 이상 시 제공발급월+1개월까지는 전월 이용금액 30만원 미만 시에도 제공###[할인 제외 대상]무이자할부, 삼성카드 할인이 적용된 일시불 및 할부 이용금액, 기프트/선불카드(포인트, 사이버머니 등 전자지급수단 포함) 구매 및 충전, 상품권 구매건 
<교통·이동통신·스트리밍 10% 결제일할인>[서비스안내]교통·이동통신·스트리밍 10% 결제일할인###[대상점]업종 할인 대상교통대중교통(버스, 지하철), 택시이동통신SKT, KT, LG U+, 알뜰폰 이동통신요금 정기결제건스트리밍넷플릭스, 웨이브, 티빙, 왓챠, 멜론, FLO 정기결제건###[할인기준]전월 이용금액대별 통합 월 할인한도를 확인할 수 있는 표입니다.30만원 이상60만원 이상10,000원20,000원###[이용조건]발급월+1개월까지는 전월 이용금액 30만원 미만 시에도 30만원 이상~60만원 미만 실적구간 혜택 제공(전월 이용금액 60만원 이상 시에는 해당 실적구간 혜택 제공)###[할인 제외 대상]무이자할부, 삼성카드 할인이 적용된 일시불 및 할부 이용금액, 교통·이동통신·스트리밍 10% 할인이 적용되지 않은 대중교통 및 택시 이용금액, 고속버스(차내 단말

In [57]:
data = {"card_name" : card_names, "card_img": card_imgs, "card_summary": card_summary,"card_benefit":card_benefit}
df = pd.DataFrame(data)

df.to_csv("samsung_cardBenfits.csv", encoding = "utf-8-sig")